In [ ]:
!pip install langchain
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "..."

LangChainによる基本的なChatボット

In [ ]:
from langchain.prompts import (
ChatPromptTemplate,
MessagesPlaceholder,
SystemMessagePromptTemplate,
HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
prompt = ChatPromptTemplate.from_messages([
  SystemMessagePromptTemplate.from_template("""
  以下は、人間とAIのフレンドリーな会話である。
  AIは饒舌で、その文脈から具体的な内容をたくさん教えてくれます。
  AIは質問の答えを知らない場合、正直に「知らない」と答えます。
  """),
  MessagesPlaceholder(variable_name="history"),
  HumanMessagePromptTemplate.from_template("{input}")
])
llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)
while True:
  command = input(">")
  if command == "q":
    break
  response = conversation.predict(input=command)
  print(f"AI:{response}")

SQL問い合わせの方法

In [ ]:
import sqlite3

# データベース接続を作成する
conn = sqlite3.connect('user_support.db')

# カーソルオブジェクトを作成する
cursor = conn.cursor()

# usersテーブルを作成する
cursor.execute('''
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT
)
''')

# productsテーブルを作成する
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    price INTEGER
)
''')

# order_historyテーブルを作成する
cursor.execute('''
CREATE TABLE order_history (
    history_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product_id INTEGER,
    purchase_date DATE,
    quantity INTEGER,
    remarks TEXT,
    created_at DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')

# support_historyテーブルを作成する
cursor.execute('''
CREATE TABLE support_history (
    history_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    subject TEXT,
    message_content TEXT,
    message_type TEXT,
    status TEXT,
    created_at DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
)
''')

# 変更をコミットする
conn.commit()

# データベース接続を閉じる
conn.close()

In [ ]:
import sqlite3
from datetime import datetime

# データベース接続を作成する
conn = sqlite3.connect('user_support.db')

# カーソルオブジェクトを作成する
cursor = conn.cursor()

# usersテーブルに日本語のテストデータを挿入する
users_data = [
    (1, '太郎', '山田', 'taro@example.com', '090-1234-5678'),
    (2, '花子', '佐藤', 'hanako@example.com', '080-9876-5432')
]

for user in users_data:
    cursor.execute('''
    INSERT INTO users (user_id, first_name, last_name, email, phone)
    VALUES (?, ?, ?, ?, ?)
    ''', user)

# productsテーブルに日本語のテストデータを挿入する
products_data = [
    (1, '商品A', 1000),
    (2, '商品B', 2000),
    (3, '商品C', 3000)
]

for product in products_data:
    cursor.execute('''
    INSERT INTO products (product_id, product_name, price)
    VALUES (?, ?, ?)
    ''', product)

# order_historyテーブルに日本語のテストデータを挿入する
order_history_data = [
    (1, 1, 1, '2023-04-01', 2, '迅速な発送', datetime.now()),
    (2, 2, 3, '2023-04-05', 1, 'ギフトラッピング', datetime.now())
]

for order in order_history_data:
    cursor.execute('''
    INSERT INTO order_history (history_id, user_id, product_id, purchase_date, quantity, remarks, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', order)

# support_historyテーブルに日本語のテストデータを挿入する
support_history_data = [
    (1, 1, '請求に関する問題', None, None, 'open', datetime.now()),
    (2, 1, None, '請求に問題があります。', 'user', None, datetime.now()),
    (3, 1, None, 'お問い合わせいただきありがとうございます。問題を調査しています。', 'support', None, datetime.now())
]

for support in support_history_data:
    cursor.execute('''
    INSERT INTO support_history (history_id, user_id, subject, message_content, message_type, status, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', support)

# 変更をコミットする
conn.commit()

# データベース接続を閉じる
conn.close()

In [ ]:
from langchain import OpenAI
from langchain import SQLDatabase
from langchain import SQLDatabaseChain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
sql_uri = "sqlite:///user_support.db"
db= SQLDatabase.from_uri(sql_uri)
llm = ChatOpenAI(temperature=0.2)
template= \
"""
ユーザテーブルのみを対象とします。
回答はJSON形式で表示してください:[{question}]
例:
[{{
"last_mame": "",
"first_name": "",
"phone": "",
"email": ""
}}]
"""
prompt = PromptTemplate(template=template, input_variables=["question"])
db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True,output_key="Answer")
user = db_chain.run(prompt.format(question="すべてのユーザを教えて"))
print(user)



> Entering new SQLDatabaseChain chain...

ユーザテーブルのみを対象とします。
回答はJSON形式で表示してください:[すべてのユーザを教えて]
例:
[{
"last_mame": "",
"first_name": "",
"phone": "",
"email": ""
}]

SQLQuery:SELECT last_name as "last_mame", first_name as "first_name", phone as "phone", email as "email" FROM users
SQLResult: [('山田', '太郎', '090-1234-5678', 'taro@example.com'), ('佐藤', '花子', '080-9876-5432', 'hanako@example.com')]
Answer:[{
"last_mame": "山田",
"first_name": "太郎",
"phone": "090-1234-5678",
"email": "taro@example.com"
},
{
"last_mame": "佐藤",
"first_name": "花子",
"phone": "080-9876-5432",
"email": "hanako@example.com"
}]
> Finished chain.
[{
"last_mame": "山田",
"first_name": "太郎",
"phone": "090-1234-5678",
"email": "taro@example.com"
},
{
"last_mame": "佐藤",
"first_name": "花子",
"phone": "080-9876-5432",
"email": "hanako@example.com"
}]


In [ ]:
import json

sql_uri = "sqlite:///user_support.db"
db= SQLDatabase.from_uri(sql_uri)
llm = ChatOpenAI(temperature=0.2)

def find_user(user_text):
  template= \
"""
ユーザテーブルのみを対象とします。次の要求文に対しユーザを一意に特定したい。
回答は例にしめすようにJSON形式で表示してください:要求文[{question}]

例:
[{{
    "user_id": ,
    "last_mame": "",
    "first_name": "",
    "phone": "",
    "email": ""
}}]
"""
  prompt = PromptTemplate(template=template, input_variables=["question"])
  db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True,output_key="Answer")
  return db_chain.run(prompt.format(question=user_text))

if __name__ == "__main__":
  print("\nお客様の情報を確認します。お名前、電話番号、ユーザID、メールなどお客様を特定できるデータを入力してください。")
  while True:
    user_text=input(">")
    user_json = find_user(user_text)
    try:
      users  = json.loads(user_json)
    except:
      print("お客様の情報が確認できませんでした。もう一度入力してください。")
      continue

    if users is None:
      print("お客様の情報が確認できませんでした。もう一度入力してください。")
      continue
    if len(users) > 1 :
      print("お客様の情報が特定できませんでした。もう一度入力してください。")
      continue

    print("お客様の情報が確認できました。")
    break
  user = '\n'.join(f'{key}: {value}' for key, value in users[0].items())
  #print(user)


お客様の情報を確認します。お名前、電話番号、ユーザID、メールなどお客様を特定できるデータを入力してください。
>ID=1


> Entering new SQLDatabaseChain chain...

ユーザテーブルのみを対象とします。次の要求文に対しユーザを一意に特定したい。
回答は例にしめすようにJSON形式で表示してください:要求文[ID=1]

例:
[{
    "user_id": ,
    "last_mame": "",
    "first_name": "",
    "phone": "",
    "email": ""
}]

SQLQuery:SELECT user_id, last_name, first_name, phone, email FROM users WHERE user_id = 1
SQLResult: [(1, '山田', '太郎', '090-1234-5678', 'taro@example.com')]
Answer:[{
    "user_id": 1,
    "last_name": "山田",
    "first_name": "太郎",
    "phone": "090-1234-5678",
    "email": "taro@example.com"
}]
> Finished chain.
お客様の情報が確認できました。


In [ ]:
from langchain.chains import SQLDatabaseSequentialChain
def get_user_info(user):

  template= \
  """
  次の要求文のユーザIDの商品名や価格も含めた購買履歴（テーブル）の内容を知りたい。
  なお回答はヘッダー付のCSV形式で出力してください。:
  要求文[{user}]
  """
  prompt = PromptTemplate(template=template, input_variables=["user"])
  chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)
  order_history = chain.run(prompt.format(user=user))

  template= \
  """
  次の要求文のユーザIDの対応履歴（テーブル）の内容を知りたい。
  なお回答はヘッダー付のCSV形式で出力してください。:
  要求文[{user}]
  """
  prompt = PromptTemplate(template=template, input_variables=["user"])
  chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)
  support_history = chain.run(prompt.format(user=user))

  return f"購買履歴:\n{order_history}\n\n対応履歴:\n{support_history}\n\n"

if __name__ == "__main__":
  user_info = get_user_info(user)



> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['users', 'order_history']

> Entering new SQLDatabaseChain chain...

  次の要求文のユーザIDの商品名や価格も含めた購買履歴（テーブル）の内容を知りたい。
  なお回答はヘッダー付のCSV形式で出力してください。:
  要求文[user_id: 1
last_name: 山田
first_name: 太郎
phone: 090-1234-5678
email: taro@example.com]
  
SQLQuery:SELECT order_history.history_id, users.first_name, users.last_name, users.email, users.phone, products.product_name, products.price, order_history.purchase_date, order_history.quantity, order_history.remarks
FROM order_history
JOIN users ON order_history.user_id = users.user_id
JOIN products ON order_history.product_id = products.product_id
WHERE users.user_id = 1
ORDER BY order_history.purchase_date DESC
LIMIT 5;
SQLResult: [(1, '太郎', '山田', 'taro@example.com', '090-1234-5678', '商品A', 1000, '2023-04-01', 2, '迅速な発送')]
Answer:ヘッダー付のCSV形式での回答:
history_id,first_name,last_name,email,phone,product_name,price,purchase_date,quantity,remarks
1,太郎,山田,taro@example.com,090-1234-5678

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

template=\
"""
        ## ユーザ情報
          {user}
          {user_info}
        ## 処理
        あなたは接客のエキスパートです。お客様の要求文にたいして的確に答えてください。
        接客のさい、上記のユーザ情報を参照し親切丁寧にお客様サポートをしてください。
        ただし、わからないものに関してはわからないと答えてください。
"""

system_template = template.format(user=user,user_info=user_info)
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0.5)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

while True:
  command = input("質問をどうぞ(qで終了)>")

  if command == "q":
    break

  response = conversation.predict(input=command)
  print(f"{response}\n")

質問をどうぞ(qで終了)>私が誰かわかりますか？
はい、お名前は山田太郎さんということでお間違いないでしょうか？

質問をどうぞ(qで終了)>そうです。私が買った商品の名前はなんでしか？


おっしゃる通り、山田太郎さんは商品Aを2個ご購入されました。

質問をどうぞ(qで終了)>いつごろ発送しました？
お調べしたところ、商品Aは2023年4月1日に購入され、同日に迅速な発送が行われました。

質問をどうぞ(qで終了)>請求の問題はどうなりました？
申し訳ありませんが、私には請求に関する情報はありません。もし請求に関する問題がある場合は、販売元に直接お問い合わせいただくことをお勧めします。

質問をどうぞ(qで終了)>q
